In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from pathlib import Path

# Загрузка данных
train_df = pd.read_csv("../../data/raw/train.csv")
test_df = pd.read_csv("../../data/raw/test.csv")

X = train_df.drop(['id', 'FloodProbability'], axis=1)
y = train_df['FloodProbability']

# Разделение на train/validation
X_train, X_val, y_train, y_val = train_test_split(
    X, y, 
    test_size=0.2,
    random_state=69,
    shuffle=True
)

# Подготовка test данных
X_test = test_df.drop('id', axis=1)
test_ids = test_df['id']

# Сохраняем датасет
output_dir = Path("../../data/processed")
output_dir.mkdir(parents=True, exist_ok=True)

X_train.to_csv(output_dir / "X_train.csv", index=False)
X_val.to_csv(output_dir / "X_val.csv", index=False)
y_train.to_csv(output_dir / "y_train.csv", index=False)
y_val.to_csv(output_dir / "y_val.csv", index=False)
X_test.to_csv(output_dir / "X_test.csv", index=False)
pd.DataFrame({'id': test_ids}).to_csv(output_dir / "test_ids.csv", index=False)